### Projeto Big Data - Predição de Risco de Crédito

#### Introdução 

Neste projeto, estamos enfrentando o desafio de prever possíveis inadimplentes em nosso produto de empréstimos ao consumidor.
Com base em dados históricos sobre o comportamento do cliente, coletados ao longo do tempo, nosso objetivo é identificar quem apresenta maior risco e quem apresenta menor risco no momento da aquisição de novos clientes.

### Dataset

O conjunto de dados inclui um total de 11 características, com uma variável alvo "risk_flag" que indica se houve ou não inadimplência no passado. O problema é uma classificação binária, onde o conjunto de treinamento possui 252.000 observações sem valores ausentes, e o conjunto de teste possui 28.000 observações com a variável alvo "risk_flag" em falta.

### Objetivo

Nosso desafio é utilizar o conjunto de treinamento para identificar padrões que possam prever a inadimplência e aplicar esses padrões ao conjunto de teste para identificar "potenciais" inadimplentes.
Vamos explorar o conjunto de dados e aplicar técnicas de aprendizado de máquina para abordar esse desafio de previsão de risco de crédito.

### Target e Features do Dataset

Claro, aqui está um resumo das colunas do conjunto de dados, explicando cada uma delas em português:

- ID: Este é o identificador único de cada cliente no conjunto de dados. Cada cliente possui um ID exclusivo que é usado para rastrear suas informações.

- Renda (Income): Esta coluna representa a renda do cliente no momento da solicitação de empréstimo. A renda é um fator importante na análise de crédito, pois influencia a capacidade do cliente de pagar o empréstimo.

- Idade (Age): A idade do cliente é a sua idade no momento da solicitação do empréstimo. A idade pode afetar a análise de crédito, pois pode indicar a estabilidade financeira e a capacidade de pagamento ao longo do tempo.

- Experiência (Experience): Esta coluna indica a experiência profissional do cliente em anos. A experiência pode ser relevante para avaliar a estabilidade no emprego e a capacidade de pagamento.

- Estado Civil (Married): Indica se o cliente é casado (Married) ou não. O estado civil pode influenciar a análise de crédito, pois pode afetar a renda disponível e as despesas familiares.

- Propriedade da Casa (House Ownership): Esta coluna descreve o tipo de propriedade da casa do cliente, como "Própria" (Own), "Alugada" (Rented) ou "Outro" (Other).

- Posse de Carro (Car Ownership): Indica se o cliente possui um carro (Yes) ou não (No). A posse de carro pode ser um fator na avaliação do risco de crédito.

- Profissão (Profession): Descreve a ocupação ou profissão do cliente. A profissão pode ser relevante para avaliar a estabilidade no emprego e a renda.

- Cidade (City): Refere-se à cidade onde o cliente reside. A cidade pode ser um fator relevante, pois os custos de vida e a demanda por crédito podem variar de acordo com a localização.

- Estado (State): Indica o estado ou região em que o cliente vive. O estado também pode influenciar os padrões de vida e a disponibilidade de crédito.

- Anos no Emprego Atual (Current Job Years): Representa o número de anos que o cliente tem no emprego atual. Isso pode ser relevante para avaliar a estabilidade no emprego.

- Anos na Residência Atual (Current House Years): Indica o número de anos que o cliente tem na residência atual. Isso pode ser relevante para avaliar a estabilidade de residência.

- Flag de Risco (Risk Flag): Esta é a variável alvo do problema. Indica se o cliente representa um risco de inadimplência (1) ou não (0). É o que queremos prever com base nas outras características.

Essas são as principais colunas do conjunto de dados e suas respectivas explicações em português. Cada uma dessas características será considerada na análise de crédito para determinar o risco associado a cada cliente.

### Imports necessarios

In [ ]:
from dask.distributed import Client, LocalCluster
import dask.dataframe as dd


In [12]:
# client.close()
client = Client(n_workers=2, threads_per_worker=2, memory_limit='0.75GB')

c:\Users\Gabriel\anaconda3\lib\site-packages\distributed\node.py:179: UserWarning: Port 8787 is already in use.
Perhaps you already have a cluster running?
Hosting the HTTP server on port 60482 instead
  warnings.warn(


In [19]:
# Leitura dos dados
train_data = dd.read_csv('Training_Data.csv')
test_data = dd.read_csv('Test_Data.csv')


In [20]:
train_data.compute()

,Id,income,age,experience,married,house_ownership,car_ownership,profession,city,state,current_job_years,current_house_years,risk_flag
0,1,1303835,23,3,single,rented,no,Mechanical_engineer,Rewa,Madhya_Pradesh,3,13,0
1,2,7574516,40,10,single,rented,no,Software_Developer,Parbhani,Maharashtra,9,13,0
2,3,3991815,66,4,married,rented,no,Technical_writer,Alappuzha,Kerala,4,10,0
3,4,6256451,41,2,single,rented,yes,Software_Developer,Bhubaneswar,Odisha,2,12,1
4,5,5768871,47,11,single,rented,no,Civil_servant,Tiruchirappalli[10],Tamil_Nadu,3,14,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...
251995,251996,8154883,43,13,single,rented,no,Surgeon,Kolkata,West_Bengal,6,11,0
251996,251997,2843572,26,10,single,rented,no,Army_officer,Rewa,Madhya_Pradesh,6,11,0
251997,251998,4522448,46,7,single,rented,no,Design_Engineer,Kalyan-Dombivli,Maharashtra,7,12,0
251998,251999,6507128,45,0,single,rented,no,Graphic_Designer,Pondicherry,Puducherry,0,10,0


In [21]:
test_data.compute()

,id,income,age,experience,married,house_ownership,car_ownership,profession,city,state,current_job_years,current_house_years
0,1,7393090,59,19,single,rented,no,Geologist,Malda,West Bengal,4,13
1,2,1215004,25,5,single,rented,no,Firefighter,Jalna,Maharashtra,5,10
2,3,8901342,50,12,single,rented,no,Lawyer,Thane,Maharashtra,9,14
3,4,1944421,49,9,married,rented,yes,Analyst,Latur,Maharashtra,3,12
4,5,13429,25,18,single,rented,yes,Comedian,Berhampore,West Bengal,13,11
...,...,...,...,...,...,...,...,...,...,...,...,...
27995,27996,9955481,57,13,single,rented,no,Statistician,Eluru[25],Andhra Pradesh,5,10
27996,27997,2917765,47,9,single,rented,no,Technical writer,Ratlam,Madhya Pradesh,9,14
27997,27998,8082415,24,5,single,rented,no,Lawyer,Mira-Bhayandar,Maharashtra,4,13
27998,27999,9474180,51,13,single,rented,yes,Chartered Accountant,Bhilai,Chhattisgarh,13,14


### Data Cleaning

#### Checking Missing Values

In [22]:
missing_values=train_data.isnull().sum().compute()
missing_values

Id                     0
income                 0
age                    0
experience             0
married                0
house_ownership        0
car_ownership          0
profession             0
city                   0
state                  0
current_job_years      0
current_house_years    0
risk_flag              0
dtype: int64

### Checking Duplicated Values

In [25]:
# Remoção de linhas duplicadas
train_data_no_duplicates = train_data.drop_duplicates()

# Comparação do número de linhas para identificar quantas foram removidas
original_row_count = train_data.shape[0].compute()
new_row_count = train_data_no_duplicates.shape[0].compute()

# Exibição do número de linhas duplicadas
print(f'Número total de linhas duplicadas: {original_row_count - new_row_count}')

Número total de linhas duplicadas: 0


### Identificando Outliers

In [30]:
# Defina a função para tratar outliers usando IQR
def remove_outliers(partition):
    Q1 = partition['income'].quantile(0.25)
    Q3 = partition['income'].quantile(0.75)
    IQR = Q3 - Q1
    
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR
    
    return partition[(partition['income'] >= lower_bound) & (partition['income'] <= upper_bound)]


In [32]:
# Aplique a função em cada partição usando map_partitions
train_data_no_outliers = train_data.map_partitions(remove_outliers)

# Compute() para obter o resultado final
train_data_no_outliers = train_data_no_outliers.compute()
train_data_no_outliers

,Id,income,age,experience,married,house_ownership,car_ownership,profession,city,state,current_job_years,current_house_years,risk_flag
0,1,1303835,23,3,single,rented,no,Mechanical_engineer,Rewa,Madhya_Pradesh,3,13,0
1,2,7574516,40,10,single,rented,no,Software_Developer,Parbhani,Maharashtra,9,13,0
2,3,3991815,66,4,married,rented,no,Technical_writer,Alappuzha,Kerala,4,10,0
3,4,6256451,41,2,single,rented,yes,Software_Developer,Bhubaneswar,Odisha,2,12,1
4,5,5768871,47,11,single,rented,no,Civil_servant,Tiruchirappalli[10],Tamil_Nadu,3,14,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...
251995,251996,8154883,43,13,single,rented,no,Surgeon,Kolkata,West_Bengal,6,11,0
251996,251997,2843572,26,10,single,rented,no,Army_officer,Rewa,Madhya_Pradesh,6,11,0
251997,251998,4522448,46,7,single,rented,no,Design_Engineer,Kalyan-Dombivli,Maharashtra,7,12,0
251998,251999,6507128,45,0,single,rented,no,Graphic_Designer,Pondicherry,Puducherry,0,10,0


### Exploratory Data Analysis